In [12]:
import io
import re
import gzip

In [34]:
myfastq = 'Z:/Users/Daniel/Test/TestProject/PreprocessTrimFilter/Mytest/cell1.fastq'

In [37]:
mynewfastq = open('Z:/Users/Daniel/Test/TestProject/PreprocessTrimFilter/Mytest/new1.fastq', 'w')

In [26]:
polyA_motif = re.compile(r'^([ACGTN]*?[CGTN])([A]{9,}[ACGNT]*$)')

In [27]:
def open_by_magic(filename):
    '''
    Adapted from : http://stackoverflow.com/questions/18367511/how-do-i-automatically-handle-decompression-when-reading-a-file-in-python
    with modifications
    Uses the initial bytes of a file to detect the file compression.

    :param str filename: path to the input file
    :return: the appropriate file handle for reading
    :rtype: file object
    '''

    ## Add more magic strs here for various compressions
    magic_dict = {"\x1f\x8b\x08":gzip.open}
    max_len = max(len(x) for x in magic_dict)
    with open(filename) as f:
        file_start = f.read(max_len)
        for magic,fn in magic_dict.items():
            if file_start.startswith(magic):
                return io.BufferedReader(fn(filename))
            return open(filename,'r') ## Otherwise just a regular file

In [28]:
def iterate_fastq(fastq):
    '''
    Read a fastq file and return the 4 lines as a list
    '''
    with open_by_magic(fastq) as IN:
        while len(IN.readline().strip()) != 0:
            yield [IN.readline().rstrip('\n'),IN.readline().rstrip('\n'),
                   IN.readline().rstrip('\n'),IN.readline().rstrip('\n')]

In [29]:
def write_fastq(read_info,OUT):
    ''' Write a fastq file
    :param str read_info: a list whose elements are the 4 lines of a fastq
    :param fastq_loc: full path to the fastq file to write to
    :return: nothing
    '''

    #out = '\n'.join(read_info)
    OUT.write(read_info+'\n')

In [30]:
def trim_read(read,seq_pattern):
    match = seq_pattern.match(read)
    if match:
        trimmed_seq = match.group(1)
        tail = match.group(2)
        #print "Trimming {polyT} from the read {read}".format(polyT=tail,read=read)
        return (trimmed_seq,match.start(2))
    else:
        return (read,0)

In [31]:
def trimfile(fastq,newfastq):
    '''
    Iterate over the R1 fastq,3' polyA tail and write as a fastq file
    '''
    for read_id,seq,p,qual in iterate_fastq(fastq):
        trimmed_seq,trimming_index = trim_read(seq,polyA_motif)
        if trimming_index: ## Non zero trimming index
            qual = qual[0:trimming_index]
        #if len(trimmed_seq) < 25: ## Drop reads below 25 b.p
        #    reads_dropped_size+=1
        #    continue
        read_info = read_id+'\n'+trimmed_seq+'\n'+p+'\n'+qual
        write_fastq(read_info,newfastq)

In [36]:
def trimfile(fastq,newfastq):
    '''
    Iterate over the R1 fastq,3' polyA tail and write as a fastq file
    '''
    for read_id,seq,p,qual in iterate_fastq(fastq):
        trimmed_seq,trimming_index = trim_read(seq,polyA_motif)
        if trimming_index: ## Non zero trimming index
            qual = qual[0:trimming_index]
        if len(trimmed_seq) < 25: ## Drop reads below 25 b.p
            reads_dropped+=1
            continue
        read_info = read_id+'\n'+trimmed_seq+'\n'+p+'\n'+qual
        write_fastq(read_info,newfastq)

In [38]:
trimfile(myfastq,mynewfastq)

UnboundLocalError: local variable 'reads_dropped' referenced before assignment

In [72]:
open_by_magic(myfastq)

<_io.TextIOWrapper name='Z:/Users/Daniel/Support/TestCase/3UPX/mySTARRun2/STAR_AS_fastq/1ng-2_S1_cell1.fastq' mode='r' encoding='cp1252'>

In [73]:
IN = open_by_magic(fastq)

In [80]:
IN.readline().strip() != 0

True

In [17]:
mynewfastq.close()